## Exercises

#### Data Acquisition

These exercises should go in a notebook or script named wrangle. Add, commit, and push your changes.

This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

Read the case, department, and source data into their own spark dataframes.

Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

- Inspect your folder structure. What do you notice?

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

- How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

- How many Stray Animal cases are there?

- How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

- Convert the council_district column to a string column.

- Extract the year from the case_closed_date column.

- Convert num_days_late from days to hours in new columns num_hours_late.

- Join the case data with the source and department data.

- Are there any cases that do not have a request source?

- What are the top 10 service request types in terms of number of requests?

- What are the top 10 service request types in terms of average days late?

- Does number of days late depend on department?

- How do number of days late depend on department and request type?

- You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.

In [1]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

import env

In [2]:
def get_db_url(db, user=env.user, host=env.host, password=env.password):
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')

In [3]:
spark = SparkSession.builder.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 15:59:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/21 15:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/21 15:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/21 15:59:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
url = get_db_url('311_data')
query = '''select * from cases c
            left join dept d on c.dept_division = d.dept_division
            left join source s on c.source_id = s.source_id;'''

In [5]:
pandas_df = pd.read_sql(query, url)

In [6]:
pandas_df.head()

,case_id,case_opened_date,case_closed_date,SLA_due_date,case_late,num_days_late,case_closed,dept_division,service_request_type,SLA_days,...,source_id,request_address,council_district,dept_division,dept_name,standardized_dept_name,dept_subject_to_SLA,index,source_id,source_username
0,1014127332,1/1/18 0:42,1/1/18 12:29,9/26/20 0:42,NO,-998.508762,YES,Field Operations,Stray Animal,999.000000,...,svcCRMLS,"2315 EL PASO ST, San Antonio, 78207",5,Field Operations,Animal Care Services,Animal Care Services,YES,132,svcCRMLS,svcCRMLS
1,1014127333,1/1/18 0:46,1/3/18 8:11,1/5/18 8:30,NO,-2.012604,YES,Storm Water,Removal Of Obstruction,4.322222,...,svcCRMSS,"2215 GOLIAD RD, San Antonio, 78223",3,Storm Water,Trans & Cap Improvements,Trans & Cap Improvements,YES,133,svcCRMSS,svcCRMSS
2,1014127334,1/1/18 0:48,1/2/18 7:57,1/5/18 8:30,NO,-3.022338,YES,Storm Water,Removal Of Obstruction,4.320729,...,svcCRMSS,"102 PALFREY ST W, San Antonio, 78223",3,Storm Water,Trans & Cap Improvements,Trans & Cap Improvements,YES,133,svcCRMSS,svcCRMSS
3,1014127335,1/1/18 1:29,1/2/18 8:13,1/17/18 8:30,NO,-15.011481,YES,Code Enforcement,Front Or Side Yard Parking,16.291887,...,svcCRMSS,"114 LA GARDE ST, San Antonio, 78223",3,Code Enforcement,Code Enforcement Services,DSD/Code Enforcement,YES,133,svcCRMSS,svcCRMSS
4,1014127336,1/1/18 1:34,1/1/18 13:29,1/1/18 4:34,YES,0.372164,YES,Field Operations,Animal Cruelty(Critical),0.125000,...,svcCRMSS,"734 CLEARVIEW DR, San Antonio, 78228",7,Field Operations,Animal Care Services,Animal Care Services,YES,133,svcCRMSS,svcCRMSS


In [7]:
cases = spark.createDataFrame(pandas_df)

ValueError: name already used as a name or title